In [40]:
import re
import numpy as np
import pandas as pd
import nltk
import torch
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, BertModel

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [67]:
df = pd.read_csv('Dataset/DataScientist.csv')
df2 = pd.read_csv('Dataset/DataAnalyst.csv')
df = df.drop(columns=['index'])
df = df.drop(columns=['Unnamed: 0'])
df2 = df2.drop(columns=['Unnamed: 0'])

df_final = pd.concat([df, df2], ignore_index=True).drop_duplicates()
df_final = df_final.drop(columns=['Easy Apply'])
print(df_final.shape)
df_final.head()

(6161, 14)


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1
1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1
2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1
3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades"
4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5"


In [68]:
print(df_final['Job Description'].apply(type).value_counts())

Job Description
<class 'str'>    6161
Name: count, dtype: int64


In [74]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


def nettoyer(text):
  if isinstance(text, str):
    text = text.lower()
    # Supprimer les caractères spéciaux
    text = re.sub(r'(http\S+|[^a-zA-Z\s])', '', text)
    # Tokenisation
    words = nltk.word_tokenize(text)
    # Suppression des mots vides et lemmatisation
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    text = ' '.join(words)
    # Reconstituer le texte prétraité
    return text
  else:
    return np.nan

df_final['Job Description'] = df_final['Job Description'].apply(nettoyer)
df_final['Company Name'] = df_final['Company Name'].apply(nettoyer)
df_final['Revenue'].replace(-1, np.nan)
df_final['Founded'].replace(-1, np.nan)
df_final.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Senior Data Scientist,$111K-$181K (Glassdoor est.),hopper hopper mission make booking travel fast...,3.5,hopper,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1
1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),noom use scientifically proven method help use...,4.5,noom u,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1
2,Data Science Manager,$111K-$181K (Glassdoor est.),decodem data science manager job description h...,-1.0,decodem,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1
3,Data Analyst,$111K-$181K (Glassdoor est.),sapphire digital seek dynamic driven midlevel ...,3.4,sapphire digital,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades"
4,"Director, Data Science",$111K-$181K (Glassdoor est.),director data science description edelman inte...,3.4,united entertainment group,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5"


In [17]:
from datasets import load_dataset
from transformers import BertTokenizerFast

# Load the CoNLL-2003 dataset
dataset = load_dataset("conll2003",trust_remote_code=True)
print(dataset)

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [ ]:
# Check dataset structure
for i in range(5):  # Check a few samples
    print("Tokens:", dataset["train"]["tokens"][i])
    print("NER Tags:", dataset["train"]["ner_tags"][i])


Tokens: ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
NER Tags: [3, 0, 7, 0, 0, 0, 7, 0, 0]
Tokens: ['Peter', 'Blackburn']
NER Tags: [1, 2]
Tokens: ['BRUSSELS', '1996-08-22']
NER Tags: [5, 0]
Tokens: ['The', 'European', 'Commission', 'said', 'on', 'Thursday', 'it', 'disagreed', 'with', 'German', 'advice', 'to', 'consumers', 'to', 'shun', 'British', 'lamb', 'until', 'scientists', 'determine', 'whether', 'mad', 'cow', 'disease', 'can', 'be', 'transmitted', 'to', 'sheep', '.']
NER Tags: [0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tokens: ['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']
NER Tags: [5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [54]:
from transformers import BertForTokenClassification

# Define the label map
label_list = ["O", "B-PER", "I-PER", "B-LOC", "I-LOC", "B-ORG", "I-ORG"]

# Load pre-trained BERT model
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [87]:
def tokenize_and_align_labels_fixed_length(dataset, tokenizer=tokenizer, max_length=50):
    # Tokenize the input text with truncation and padding to a fixed length
    tokenized_inputs = tokenizer(
        dataset["tokens"],
        truncation=True,
        padding="max_length",
        max_length=max_length,
        is_split_into_words=True,
        return_overflowing_tokens=False,  # Prevent splitting long sequences
    )
    
    aligned_labels = []
    for i, labels in enumerate(dataset["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to words
        label_ids = []
        previous_word_idx = None

        for word_idx in word_ids:
            if word_idx is None:
                # Special tokens (e.g., [CLS], [SEP], or padding)
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # Assign label for the first sub-token of a word
                label_ids.append(labels[word_idx])
            else:
                # Sub-token should be masked with -100
                label_ids.append(-100)
            previous_word_idx = word_idx

        # Ensure labels are padded to max_length
        if len(label_ids) < max_length:
            label_ids += [-100] * (max_length - len(label_ids))
        else:
            label_ids = label_ids[:max_length]
        
        aligned_labels.append(label_ids)

    # Update tokenized inputs with aligned labels
    tokenized_inputs["labels"] = aligned_labels

    # Force all keys to have consistent lengths
    for key in tokenized_inputs.keys():
        if key in ["input_ids", "attention_mask", "token_type_ids"]:
            tokenized_inputs[key] = [
                seq[:max_length] + [0] * (max_length - len(seq)) if len(seq) < max_length else seq[:max_length]
                for seq in tokenized_inputs[key]
            ]
    
    return tokenized_inputs


In [88]:
encoded_dataset = dataset.map(tokenize_and_align_labels_fixed_length, batched=True)

Map: 100%|██████████| 3453/3453 [00:00<00:00, 6830.37 examples/s]


In [90]:
# Check dataset structure
for i in range(5):  # Check a few samples
    print("Tokens:", encoded_dataset["train"]["tokens"][i])
    print("NER Tags:", encoded_dataset["train"]["ner_tags"][i])


Tokens: ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
NER Tags: [3, 0, 7, 0, 0, 0, 7, 0, 0]
Tokens: ['Peter', 'Blackburn']
NER Tags: [1, 2]
Tokens: ['BRUSSELS', '1996-08-22']
NER Tags: [5, 0]
Tokens: ['The', 'European', 'Commission', 'said', 'on', 'Thursday', 'it', 'disagreed', 'with', 'German', 'advice', 'to', 'consumers', 'to', 'shun', 'British', 'lamb', 'until', 'scientists', 'determine', 'whether', 'mad', 'cow', 'disease', 'can', 'be', 'transmitted', 'to', 'sheep', '.']
NER Tags: [0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tokens: ['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']
NER Tags: [5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)


# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    use_cpu=True,
    remove_unused_columns=False,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    processing_class=tokenizer,
    data_collator=data_collator
)
trainer.train()


In [ ]:
from sklearn.metrics import classification_report

# Perform evaluation on the test set
predictions, labels, _ = trainer.predict(test_data)
predictions = np.argmax(predictions, axis=2)

# Convert predictions and labels to their original format
true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
predicted_labels = [[label_list[p] for p in pred if p != -100] for pred in predictions]

# Flatten the lists for evaluation
flat_true_labels = [item for sublist in true_labels for item in sublist]
flat_predicted_labels = [item for sublist in predicted_labels for item in sublist]

# Generate classification report
print(classification_report(flat_true_labels, flat_predicted_labels))
